In [1]:
"""!conda install -c conda-forge openexr-python -y
!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir 
!pip install --upgrade google-cloud-storage"""

'!conda install -c conda-forge openexr-python -y\n!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir \n!pip install --upgrade google-cloud-storage'

In [51]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import pickle

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_metrics_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2
from google.protobuf import text_format

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')

# deactivating the warnings
import warnings
warnings.filterwarnings('ignore')

from download import download_from_gcs
from prepare_dataset import Prepare_train_dataset, Prepare_validation_dataset, Prepare_test_dataset
from data_preprocess import process_waymo_data_with_scenario_proto


# Downloader

In [46]:
# Download samples
download_from_gcs('uncompressed/scenario/validation/validation.tfrecord-00000-of-00150')
download_from_gcs('uncompressed/scenario/training/training.tfrecord-00000-of-01000')
download_from_gcs('uncompressed/scenario/testing/testing.tfrecord-00000-of-00150')

validation.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/validation/validation.tfrecord-00000-of-00150
training.tfrecord-00000-of-01000 already exists in waymo_open_dataset_/training/training.tfrecord-00000-of-01000
testing.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/testing/testing.tfrecord-00000-of-00150


# Loader

In [75]:
!python data_preprocess.py

2023-05-27 20:45:34.980413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 20:45:38.107974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-27 20:45:38.108018: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-27 20:45:45.753234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [78]:
# load scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl
pickle_file = 'scenarios/processed_scenarios_training/sample_1a8cc570d620bd31.pkl'
processed = pickle.load(open(pickle_file, 'rb'))

print('The shape of the original trajectories is: ', processed['track_infos']['trajs'].shape)
print('The shape of the tracks to predict until current time step is: ', processed['track_infos']['tracks_to_predict_until_current'].shape)
print('The shape of the tracks to predict after current time step is: ', processed['track_infos']['tracks_to_predict_future'].shape)

The shape of the original trajectories is:  (105, 91, 10)
The shape of the tracks to predict until current time step is:  (4, 11, 10)
The shape of the tracks to predict after current time step is:  (4, 80, 10)
